In [1]:
import glob
import SimpleITK as sitk
import numpy as np
import collections
from PIL import Image, ImageDraw
from segmentation import segment
import os
import csv
import collections
from collections import namedtuple 
import random
from pylidc_func import masks_build, ann_group, ann_counter
from nodule_segmentation import nodule_segment
# import pylidc

In [4]:
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

In [5]:
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr) if np.max(np_arr) > 0 else 1
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices.save(target_path)

In [4]:
# uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886"
# mhd_path = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/{}.mhd'.format(uid))

# ct_mhd = sitk.ReadImage(mhd_path)

# if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
#     ct_mhd = ct_mhd[...,0]

# hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

# origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

# vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

# direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

# mask_a, ann_count = masks_build(uid, hu_a)
# print(mask_a.sum())
# print(ann_count.sum())

In [5]:
# for i in range(hu_a.shape[0]):
#         np2Png(hu_a[i], "./test_data/origin/origin_{}.png".format(i))
#         np2Png(ann_count[i], "./test_data/count/count_{}.png".format(i))
#         np2Png(mask_a[i].astype(int), "./test_data/mask/mask_{}.png".format(i))

In [3]:
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, diameter_mm, series_uid, center_xyz')
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob('D:/LUNA/Luna16_AugData/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    candidateInfo_list = []
    with open('C:/Users/oplab/Desktop/Luna16_data/annotations_with_malignancy.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])
            isMal_bool = {'False': False, 'True': True}[row[5]] #it record the malignancy or not

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    annotationDiameter_mm,
                    series_uid,
                    annotationCenter_xyz,
                )
            )
#     with open('C:/Users/oplab/Desktop/Luna16_data/candidates_V2.csv', "r") as f:
#         for row in list(csv.reader(f))[1:]:
#             series_uid = row[0]

#             if series_uid not in presentOnDisk_set and requireOnDisk_bool:
#                 continue

#             isNodule_bool = bool(int(row[4]))
#             candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

#             if not isNodule_bool:
#                 candidateInfo_list.append(
#                     CandidateInfoTuple(
#                         False,
#                         False,
#                         False,
#                         0.0,
#                         series_uid,
#                         candidateCenter_xyz,
#                     )
#             )
#     candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [7]:
candidateInfo_list = getCandidateInfoList()
len(candidateInfo_list)

1182

In [6]:
def buildAnnotationMask(noduleInfo_list, hu_a, origin_xyz, vxSize_xyz, direction_a, threshold_hu = -700):
        boundingBox_a = np.zeros_like(hu_a, dtype=bool) # all False tensor

        for candidateInfo_tup in noduleInfo_list: #traverse所有的nodules
            center_irc = xyz2irc(
                candidateInfo_tup.center_xyz,
                origin_xyz,
                vxSize_xyz,
                direction_a,
            )
            ci = int(center_irc.index)  # the center of voxel
            cr = int(center_irc.row)
            cc = int(center_irc.col)

            index_radius = 2
            try:
                # 從index找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci + index_radius, cr, cc] > threshold_hu and \
                        hu_a[ci - index_radius, cr, cc] > threshold_hu:
                    index_radius += 1
            except IndexError:
                index_radius -= 1

            row_radius = 2
            try:
                # 從row找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr + row_radius, cc] > threshold_hu and \
                        hu_a[ci, cr - row_radius, cc] > threshold_hu:
                    row_radius += 1
            except IndexError:
                row_radius -= 1

            col_radius = 2
            try:
                # 從column找，看哪裡會遇到空氣，當兩邊都遇到空氣後就把邊界設在大的那段
                while hu_a[ci, cr, cc + col_radius] > threshold_hu and \
                        hu_a[ci, cr, cc - col_radius] > threshold_hu:
                    col_radius += 1
            except IndexError:
                col_radius -= 1

            # assert index_radius > 0, repr([candidateInfo_tup.center_xyz, center_irc, self.hu_a[ci, cr, cc]])
            # assert row_radius > 0
            # assert col_radius > 0

            boundingBox_a[
                 ci - index_radius: ci + index_radius + 1,
                 cr - row_radius: cr + row_radius + 1,
                 cc - col_radius: cc + col_radius + 1] = True #將box裡的所有格子設成TRUE

        mask_a = boundingBox_a & (hu_a > threshold_hu)  #最後會對box和threshold低於-700的值做and

        return mask_a

In [8]:
def getCandidateInfoDict(requireOnDisk_bool=True):  #把candidateInfoList包成Dict
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup)

    return candidateInfo_dict

In [9]:
candidate_dict =  getCandidateInfoDict()
len(candidate_dict)

600

In [11]:
# idx = 0
# uid_ndcnt = 0
# for uid in candidate_dict:
    
#     mhd_path = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/{}.mhd'.format(uid))

#     ct_mhd = sitk.ReadImage(mhd_path)

#     if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
#         ct_mhd = ct_mhd[...,0]
        
#     hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

#     origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

#     vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

#     direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)
    
#     mask_a= masks_build(uid, hu_a)
    
#     lower_bound = 0
    
#     upper_bound = len(hu_a)
    
#     current_slice = 0
    
#     uid_ndcnt = 0
    
#     for candidateInfo_tup in candidate_dict[uid]:
        
#         center_irc = xyz2irc(candidateInfo_tup.center_xyz, origin_xyz, vxSize_xyz, direction_a)
    
#         index, row, column = center_irc
        
#         hu_list = []
#         mask_list = []
#         for i in range(0, 1):
#         # for i in range(-3, 4):
#             current_slice = index + i
#             if index + i < lower_bound:
#                 current_slice = lower_bound
#             if index + i > upper_bound:
#                 current_slice = upper_bound
#             np2Png(hu_a[current_slice], "C:/LUNA/U-Det/Data/imgs-photo/{}-({})_{}_{}.png".format(idx, i, uid_ndcnt, uid))
#             np2Png(mask_a[current_slice].astype(float), "C:/LUNA/U-Det/Data/masks-photo/{}-({})_{}_{}.png".format(idx, i, uid_ndcnt, uid))
#             hu_list.append(hu_a[current_slice])
#             mask_list.append(mask_a[current_slice])
#         # tr_data = np.stack(hu_list)
#         # gt_data = mask_list[3]
#         # np.save('C:/LUNA/U-Det/Data/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), tr_data)
#         # np.save('C:/LUNA/U-Det/Data/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), gt_data)
#         # np.save('C:/LUNA/U-Det/Data/np_files/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), tr_data)
#         # np.save('C:/LUNA/U-Det/Data/np_files/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), gt_data)
#         np.save('C:/LUNA/U-Det/Data/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), hu_list[0])
#         np.save('C:/LUNA/U-Det/Data/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), mask_list[0])
#         np.save('C:/LUNA/U-Det/Data/np_files/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), hu_list[0])
#         np.save('C:/LUNA/U-Det/Data/np_files/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), mask_list[0])
#         idx += 1
#         uid_ndcnt += 1
#     break

In [12]:
# idx = 0
# uid_ndcnt = 0
# csv_file_path = "C:/Users/oplab/Desktop/Luna16_data/ann.csv"
# # Writing to CSV file
# with open(csv_file_path, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     ann_header = ("UID", "NID", "Start", "End", "StartRow", "EndRow", "StartCol", "EndCol", "Diameter")
#     writer.writerow(ann_header)
#     for uid in candidate_dict:

#         mhd_path = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/{}.mhd'.format(uid))

#         ct_mhd = sitk.ReadImage(mhd_path)

#         if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
#             ct_mhd = ct_mhd[...,0]

#         hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

#         origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

#         vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

#         direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

#         # mask_a= masks_build(uid, hu_a)

#         ann_data = ann_group(uid, hu_a)
#         # print(len(ann_data))
#         # counter += len(ann_data)

#         # print(ann_data)


#         # Writing to CSV file

#             # Write the header

#         # Write the data rows
#         writer.writerows(ann_data)
#         # break
        
#         # print(uid)

#         # print(f"Data has been written to {csv_file_path}")


#     #     lower_bound = 0

#     #     upper_bound = len(hu_a)

#     #     current_slice = 0

#     #     uid_ndcnt = 0

#     #     for candidateInfo_tup in candidate_dict[uid]:

#     #         center_irc = xyz2irc(candidateInfo_tup.center_xyz, origin_xyz, vxSize_xyz, direction_a)

#     #         index, row, column = center_irc

#     #         hu_list = []
#     #         mask_list = []
#     #         for i in range(0, 1):
#     #         # for i in range(-3, 4):
#     #             current_slice = index + i
#     #             if index + i < lower_bound:
#     #                 current_slice = lower_bound
#     #             if index + i > upper_bound:
#     #                 current_slice = upper_bound
#     #             np2Png(hu_a[current_slice], "C:/LUNA/U-Det/Data/imgs-photo/{}-({})_{}_{}.png".format(idx, i, uid_ndcnt, uid))
#     #             np2Png(mask_a[current_slice].astype(float), "C:/LUNA/U-Det/Data/masks-photo/{}-({})_{}_{}.png".format(idx, i, uid_ndcnt, uid))
#     #             hu_list.append(hu_a[current_slice])
#     #             mask_list.append(mask_a[current_slice])
#     #         # tr_data = np.stack(hu_list)
#     #         # gt_data = mask_list[3]
#     #         # np.save('C:/LUNA/U-Det/Data/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), tr_data)
#     #         # np.save('C:/LUNA/U-Det/Data/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), gt_data)
#     #         # np.save('C:/LUNA/U-Det/Data/np_files/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), tr_data)
#     #         # np.save('C:/LUNA/U-Det/Data/np_files/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), gt_data)
#     #         np.save('C:/LUNA/U-Det/Data/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), hu_list[0])
#     #         np.save('C:/LUNA/U-Det/Data/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), mask_list[0])
#     #         np.save('C:/LUNA/U-Det/Data/np_files/imgs/images_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), hu_list[0])
#     #         np.save('C:/LUNA/U-Det/Data/np_files/masks/masks_{}_{}_{}.npy'.format(idx, uid_ndcnt, uid), mask_list[0])
#     #         idx += 1
#     #         uid_ndcnt += 1
#         # break
# # print(counter)

In [10]:
from skimage import measure, filters
import scipy.ndimage.morphology
from os.path import join
import matplotlib.pyplot as plt
def threshold_mask(raw_output, threshold):
    if threshold == 0:
        try:
            threshold = filters.threshold_otsu(raw_output)
        except:
            threshold = 0.5

    # print('\tThreshold: {}'.format(threshold))

    raw_output[raw_output > threshold] = 1
    raw_output[raw_output < 1] = 0

    all_labels = measure.label(raw_output)
    props = measure.regionprops(all_labels)
    props.sort(key=lambda x: x.area, reverse=True)
    thresholded_mask = np.zeros(raw_output.shape)

    if len(props) >= 2:
        if props[0].area / props[1].area > 5:  # if the largest is way larger than the second largest
            thresholded_mask[all_labels == props[0].label] = 1  # only turn on the largest component
        else:
            thresholded_mask[all_labels == props[0].label] = 1  # turn on two largest components
            thresholded_mask[all_labels == props[1].label] = 1
    elif len(props):
        thresholded_mask[all_labels == props[0].label] = 1

    thresholded_mask = scipy.ndimage.morphology.binary_fill_holes(thresholded_mask).astype(np.uint8)

    return thresholded_mask
def plot_image(img_data, gt_data, output, i, outdir):
    img_name = "{}".format(i)
    output_bin = threshold_mask(output, 0.5)
    # fig_out_dir = "D://LUNA//udet//result//origin"
    fig_out_dir = "C://LUNA//udet//result//{}//plot".format(outdir)
    f, ax = plt.subplots(figsize=(10, 5))

    ax.imshow(img_data, alpha=1, cmap='gray')
    ax.imshow(output_bin, alpha=0.5, cmap='Blues')
    ax.imshow(gt_data, alpha=0.5, cmap='Reds')
    ax.axis('off')
    fig = plt.gcf()
    # fig.suptitle(img_name)

    # print("save image at: ", join(fig_out_dir, img_name + '_qual_fig' + '.png'))
    plt.savefig(join(fig_out_dir, img_name + '.png'),
                format='png', bbox_inches='tight')
    plt.close('all')
from PIL import Image, ImageDraw
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    # d = 4000 if max_value != 1 else 1
    diff = np_arr - min_value
    d = max_value - min_value
    # print(min_value)
    # print(max_value)
    if (min_value == max_value and min_value == 1000):
        diff = np_arr
        d = 1000
    scaled_np_arr = diff / (d) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices.save(target_path)

In [11]:
idx = 0
uid_ndcnt = 0
outdir = "augseg_new"
for uid in candidate_dict:
    
    uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143"

    mhd_path = glob.glob('D:/LUNA/Luna16_AugSegData/subset*/{}.mhd'.format(uid))
    
    mask_path = glob.glob('C:/LUNA/Luna16_AugMask/subset*/{}.mhd'.format(uid))
    
    mask_mhd = sitk.ReadImage(mask_path)
    
    if mask_mhd.GetDimension()==4 and mask_mhd.GetSize()[3]==1:
        mask_mhd = mask_mhd[...,0]
    mask_a = sitk.GetArrayFromImage(mask_mhd)

    ct_mhd = sitk.ReadImage(mhd_path)

    if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
        ct_mhd = ct_mhd[...,0]

    hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    # mask_a= masks_build(uid, hu_a)
    
    result = nodule_segment(hu_a)
    print(result.shape)
    for i in range(result.shape[0]):
        np2Png(hu_a[i], "C://LUNA//udet//result//{}//origin//{}_{}.png".format(outdir, i, uid))
        ori_mask = np.where(mask_a[i], hu_a[i], 1000)
        np2Png(ori_mask, "C://LUNA//udet//result//{}//ori_mask//{}_{}.png".format(outdir, i, uid))
        # break
    break

Predict: 100%|███████████████████████████████████████████████████████████████████████████| 246/246 [00:28<00:00,  8.67it/s]


(491, 512, 512)


FileNotFoundError: [Errno 2] No such file or directory: 'C://LUNA//udet//result//augseg_new//origin//0_1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143.png'

In [15]:
result.shape

(491, 512, 512)

In [16]:
mask_a.shape

(491, 512, 512)

In [12]:
for i in range(result.shape[0]):
    plot_image(hu_a[i], mask_a[i], result[i], i, outdir)

C:\Users\oplab\AppData\Local\Temp\ipykernel_20544\1730159673.py:31: DeprecationWarning: Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  thresholded_mask = scipy.ndimage.morphology.binary_fill_holes(thresholded_mask).astype(np.uint8)


In [18]:
import torch
torch.cuda.is_available()

True